In [ ]:
import streamlit as st
import pandas as pd
import snowbooks_extras

In [ ]:
use role sysadmin;
use database ai_demos;
use schema public;

In [ ]:
create or replace stage ai_demos.public.EXAMPLE_ITEMS 
	DIRECTORY = ( ENABLE = true 
                  AUTO_REFRESH = TRUE) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

In [ ]:
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

MY_STAGE = 'EXAMPLE_DOCS/images'
MY_FILE_NAME = "images/*"


# Upload the file to a stage.
put_result = session.file.put(MY_FILE_NAME, MY_STAGE, auto_compress=False,overwrite=True)


In [ ]:
alter stage EXAMPLE_ITEMS refresh;

In [ ]:
select * from directory(@example_items);

# Product Image Matching
The code below finds images that show the same or similar products

In [ ]:
-- Image matching with AI_FILTER. This is in Private Preview, so will require enablement from account team
WITH pictures AS (
  SELECT
      TO_FILE(file_url) AS img
  FROM DIRECTORY(@EXAMPLE_ITEMS)
)
Select self1.img, self2.img 
from pictures self1
join pictures self2
on AI_FILTER(Prompt('Does the product shown in {0} match the product in {1}?', self1.img, self2.img));

In [ ]:
-- This approach uses AI_SIMILARITY, generating vector embeddings for each one
WITH pictures AS (
  SELECT
    relative_path,
      TO_FILE(file_url) AS img
  FROM DIRECTORY(@EXAMPLE_ITEMS)
)
SELECT
self1.img as image_1,
self2.img as image_2,
-- Semantic similarity score. Using 'voyage-multimodal-3' model
AI_SIMILARITY(self1.img, self2.img)as sim_score,
FROM  pictures self1
CROSS JOIN pictures self2
where sim_score < 1
QUALIFY row_number() OVER (PARTITION BY self1.relative_path
                         ORDER BY sim_score DESC) <= 3; -- Top 3

# Luxury good extraction
1. Start with filtering out images that have a luxury good
2. Then get details about the item

In [ ]:
-- Create dataset of images
WITH pictures AS (
  SELECT
      TO_FILE(file_url) AS img,
      relative_path
  FROM DIRECTORY(@EXAMPLE_ITEMS)
),
-- Filter to only real estate pictures with luxury goods
luxury_goods_only as (
select *
from pictures
where ai_filter(prompt('Is this image {0} a real estate listing image with a luxury good?', pictures.img)
))
-- Extract information about that luxury good
select * from luxury_goods_only;

In [ ]:
-- Create dataset of images
WITH pictures AS (
  SELECT
      TO_FILE(file_url) AS img,
      relative_path
  FROM DIRECTORY(@EXAMPLE_ITEMS)
),
-- Filter to only real estate pictures with luxury goods
luxury_goods_only AS (
  SELECT *
  FROM pictures
  WHERE ai_filter(prompt('Is this image {0} a real estate listing image with a luxury good?', pictures.img))
),
-- Extract information about that luxury good
luxury_items_classified AS (
  SELECT 
      *,
      -- Classify the luxury item into categories
      AI_CLASSIFY(
          img,
          ['painting', 'jewelry', 'musical instrument', 'sculpture', 'antique furniture', 'watch', 'collectible', 'other']
      ) AS item_class,

      AI_EXTRACT(
        file => img,
        responseFormat => [
            ['description', Concat('Give me a brief description of the luxury good pictured in this room. It belongs to the following category: ', item_class:labels[0]::string)]
        ]
      ) as item_details
      
  FROM luxury_goods_only
)

SELECT 
    img,
    relative_path,
    item_class:labels[0]::string as item_class,
    item_details:response
FROM luxury_items_classified;